# Classificação de imagens de tomografia computadorizada de próstata

## Instalações

In [1]:
import sys

In [2]:
!{sys.executable} -m pip install pydicom

In [3]:
!{sys.executable} -m pip install pypng

In [4]:
!{sys.executable} -m pip install keras

In [5]:
!{sys.executable} -m pip install tensorflow

## Definição de paths e arquivos treino / teste

In [24]:
training_data = 'imagens/treinamento'
validation_data = 'imagens/validacao'

In [25]:
import pylab
import pydicom as dicom
import glob
import png
import os

#Transforma as imagens DICOM em PNG

diretorio_list = ['imagens/treinamento/true/', 'imagens/validacao/true/',
                  'imagens/treinamento/false/', 'imagens/validacao/false/']
for diretorio in diretorio_list:
    for img in glob.glob(diretorio + '*.dcm'):
          mri_file = open(img, 'rb')
          imagem = dicom.read_file(mri_file)
          mri_file.close()

          shape = imagem.pixel_array.shape

          image_2d = []
          max_val = 0
          for row in imagem.pixel_array:
              pixels = []
              for col in row:
                  pixels.append(col)
                  if col > max_val: max_val = col
              image_2d.append(pixels)

          image_2d_scaled = []
          for row in image_2d:
              row_scaled = []
              for col in row:
                  col_scaled = int((float(col) / float(max_val)) * 255.0)
                  row_scaled.append(col_scaled)
              image_2d_scaled.append(row_scaled)

          png_nome = os.path.basename(img)[:-3] + 'png'
          print(png_nome)
          png_file = open(diretorio+png_nome, 'wb')
          w = png.Writer(shape[1], shape[0], greyscale=True)
          w.write(png_file, image_2d_scaled)
          png_file.close()


0000-12-000007.png
0002-12-000007.png
0005-12-000007.png
0014-12-000007.png
0015-12-000007.png
0018-12-000007.png
0019-12-000007.png
0026-12-000007.png
0028-12-000007.png
0031-12-000007.png
0035-12-000007.png
0037-12-000007.png
0040-12-000007.png
0041-12-000007.png
0046-12-000007.png
0054-12-000007.png
0055-12-000007.png
0067-12-000007.png
0071-12-000007.png
0077-12-000007.png
0078-12-000007.png
0082-12-000007.png
0092-12-000007.png
0094-12-000007.png
0095-12-000007.png
0097-12-000007.png
0099-12-000007.png
0102-12-000007.png
0103-12-000007.png
0104-12-000007.png
0105-12-000007.png
0106-12-000007.png
0111-12-000007.png
0114-12-000007.png
0115-12-000007.png
0117-12-000007.png
0118-12-000007.png
0119-12-000007.png
0121-12-000007.png
0123-12-000007.png
0125-12-000007.png
0126-12-000007.png
0127-12-000007.png
0128-12-000007.png
0129-12-000007.png
0131-12-000007.png
0139-12-000007.png
0143-12-000007.png
0164-12-000007.png
0169-12-000007.png
0179-12-000007.png
0181-12-000007.png
0183-12-0000

In [26]:
import keras
import numpy as np

import os
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

In [27]:
img_width, img_height = 512, 512

# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        training_data,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')


Found 182 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


In [28]:
# Initialising the CNN
model = Sequential()

# Convolution
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional layer
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())


# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

C:\Users\Rodrigo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(512, 512,...)`
  """
C:\Users\Rodrigo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  if sys.path[0] == '':
C:\Users\Rodrigo\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


## Treinamento

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
#Definição de variáveis de treinamento

nb_epoch  = 10
nb_train_samples = 160
nb_validation_samples = 24

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)
model.save_weights('modelos/checkpoint.h5')

C:\Users\Rodrigo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
C:\Users\Rodrigo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=10, epochs=10, validation_steps=24)`
  if sys.path[0] == '':


Epoch 1/10
10/10 [==============================] - 86s 9s/step - loss: 1.8809 - acc: 0.7453 - val_loss: 0.6939 - val_acc: 0.5556
Epoch 2/10
 9/10 [==========================>...] - ETA: 7s - loss: 0.6922 - acc: 0.6898 

## Validação

In [23]:
from keras.preprocessing import image

img = image.load_img('imagens/validacao/true/0164-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0164 (Com tumor)',prediction);

img = image.load_img('imagens/validacao/true/0169-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0169 (Com tumor)',prediction);

img = image.load_img('imagens/validacao/true/0179-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0179 (Com tumor)',prediction);

img = image.load_img('imagens/validacao/true/0181-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0181 (Com tumor)',prediction);

img = image.load_img('imagens/validacao/true/0183-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0183 (Com tumor)',prediction);

img = image.load_img('imagens/validacao/true/0184-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0184 (Com tumor)',prediction);

img = image.load_img('imagens/validacao/true/0186-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0186 (Com tumor)',prediction);

img = image.load_img('imagens/validacao/false/0161-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0161 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0162-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0162 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0163-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0163 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0165-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0165 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0166-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0166 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0167-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0161 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0168-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0168 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0170-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0170 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0171-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0171 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0172-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0172 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0173-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0173 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0174-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0174 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0175-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0175 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0176-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0176 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0177-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0177 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0178-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0178 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0180-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0180 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0182-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0182 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0184-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0184 (Sem tumor)',prediction);

img = image.load_img('imagens/validacao/false/0185-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('Paciente 0185 (Sem tumor)',prediction);

img = image.load_img('imagens/treinamento/true/0111-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('[Treino] Paciente 0111 (Com tumor)',prediction);

img = image.load_img('imagens/treinamento/false/0050-12-000007.png', target_size=(512, 512))
img_teste = image.img_to_array(img)
img_teste = np.expand_dims(img_teste, axis=0) * 1./255
prediction = model.predict(img_teste)
print('[Treino] Paciente 0050 (Sem tumor)',prediction);

Paciente 0164 (Com tumor) [[0.30553293]]
Paciente 0169 (Com tumor) [[0.32652175]]
Paciente 0179 (Com tumor) [[0.29831216]]
Paciente 0181 (Com tumor) [[0.30785495]]
Paciente 0183 (Com tumor) [[0.32773614]]
Paciente 0184 (Com tumor) [[0.30658942]]
Paciente 0186 (Com tumor) [[0.3060127]]
Paciente 0161 (Sem tumor) [[0.32209164]]
Paciente 0162 (Sem tumor) [[0.30426404]]
Paciente 0163 (Sem tumor) [[0.31410682]]
Paciente 0165 (Sem tumor) [[0.31892943]]
Paciente 0166 (Sem tumor) [[0.287325]]
Paciente 0161 (Sem tumor) [[0.34496]]
Paciente 0168 (Sem tumor) [[0.2946217]]
Paciente 0170 (Sem tumor) [[0.31167427]]
Paciente 0171 (Sem tumor) [[0.32248095]]
Paciente 0172 (Sem tumor) [[0.3056389]]
Paciente 0173 (Sem tumor) [[0.2718318]]
Paciente 0174 (Sem tumor) [[0.2853437]]
Paciente 0175 (Sem tumor) [[0.29986298]]
Paciente 0176 (Sem tumor) [[0.33838645]]
Paciente 0177 (Sem tumor) [[0.29684958]]
Paciente 0178 (Sem tumor) [[0.2918002]]
Paciente 0180 (Sem tumor) [[0.3071956]]
Paciente 0182 (Sem tumor) [[